In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import library
import torch, torchvision
from pathlib import Path
import numpy as np

import cv2
import pandas as pd
from tqdm import tqdm
import PIL.Image as Image
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.ticker import MaxNLocator
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from glob import glob
import shutil
from collections import defaultdict
import os
from torch import nn , optim
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.datasets import  ImageFolder
from torch.utils.data import DataLoader
from torchvision import models

In [ ]:
!unzip /kaggle/input/aerial-cactus-identification/train.zip

In [ ]:
!unzip /kaggle/input/aerial-cactus-identification/test.zip

In [ ]:
df_train = pd.read_csv("/kaggle/input/aerial-cactus-identification/train.csv")

In [ ]:
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train['has_cactus'].value_counts()

In [ ]:
df_sample = pd.read_csv("/kaggle/input/aerial-cactus-identification/sample_submission.csv")

In [ ]:
df_sample.shape

In [ ]:
train_path = "./train/"
test_path = "./test/"

In [ ]:
from glob import glob
train_list = glob(os.path.join(train_path,'*'))
test_list = glob(os.path.join(test_path,'*'))
print(f"Num of train {len(train_list)}")
print(f"Num of test {len(test_list)}")

In [ ]:
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
import matplotlib.pyplot as plt
#visualine image
label = 'Has_Cactus', 'No_Cactus'
plt.figure(figsize =(8,8))
plt.pie(df_train.groupby('has_cactus').size(), labels = label, autopct='%1.1f%%', shadow=True, startangle=90)
plt.show()

In [ ]:
#visualize some img

import matplotlib.image as image
fig, ax = plt.subplots(1, 5, figsize=(15,3))

for i, idx in enumerate(df_train[df_train['has_cactus']==1]['id'][:5]):
    path = os.path.join(train_path, idx)
    ax[i].imshow(image.imread(path))

In [ ]:
#visualize some img

import matplotlib.image as image
fig, ax = plt.subplots(1, 5, figsize=(15,3))

for i, idx in enumerate(df_train[df_train['has_cactus']==0]['id'][:5]):
    path = os.path.join(train_path, idx)
    ax[i].imshow(image.imread(path))

In [ ]:
# #define customer imshow function

# import numpy as np


# def imshow(img, ax=None, title = None, normalize = None):
#     if ax is None:
#         fig, ax = plt.subplots()
    
#     img = img.numpy().transpose((1,2,0))
#     if normalize:
#         mean = np.array([0.485,0.456,0.406])
#         std = np.array([0.229, 0.224,0.225])
#         img = std * img + mean
#         img = np.clip(img,0,1)
#     ax.imshow(img)
#     ax.splines['top'].set_visible(False)
#     ax.splines['right'].set_visible(False)
#     ax.splines['left'].set_visible(False)
#     ax.splines['bottom'].set_visible(False)
#     ax.tick_params(axis='both',length=0)
#     ax.set_xticklabels('')
#     ax.set_yticklabels('')
#     return ax

In [ ]:
#visualize some img

import matplotlib.image as image
fig, ax = plt.subplots(1, 5, figsize=(15,3))

for i, idx in enumerate(df_train[df_train['has_cactus']==0]['id'][:5]):
    path = os.path.join(train_path, idx)
    ax[i].imshow(image.imread(path))

In [ ]:
#class Dataset
class CustomDataset(Dataset):
    def __init__(self, df, path, transform=None):
        super().__init__()
        self.df = df
        self.path = path
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img, label = self.df.iloc[idx,0], self.df.iloc[idx,1]
        img_path = os.path.join(self.path, img)
        img = image.imread(img_path)
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [ ]:
#split train valid
train_df, valid_df = train_test_split(df_train, test_size = 0.2,random_state = 24)

In [ ]:
#transform
mean = np.array([0.485,0.456,0.406])
std = np.array([0.229, 0.224,0.225])

data_transform = {
    'train': transforms.Compose([transforms.ToPILImage(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean,std)]),
    'valid': transforms.Compose([transforms.ToPILImage(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean,std)]),
    'test' : transforms.Compose([transforms.ToPILImage(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean,std)])
}
data_frame = {
    'train' : train_df,
    'valid' : valid_df,
    'test' : df_sample
}
data_path = {
    'train' : train_path,
    'valid' : train_path,
    'test' : test_path
}

In [ ]:
dataset_ds = {x : CustomDataset(data_frame[x],data_path[x],transform=data_transform[x]) for x in ['train','valid','test']}
# train_ds = CustomDataset(train_df,train_path,transform=train_transform)
# valid_ds = CustomDataset(valid_df,train_path,transform=valid_transform)
# test_ds = CustomDataset(df_sample,test_path,transform=test_transform)

In [ ]:
data_loader = {x : DataLoader(dataset_ds[x],batch_size =32,shuffle=True) for x in ['train','valid','test'] }
data_sizes = {x : len(dataset_ds[x]) for x in ['train','valid','test'] }
class_names = ['yes','no']

In [ ]:
data_sizes

In [ ]:
#show image  by batch
def imshow(inp, title=None):
  inp = inp.numpy().transpose((1,2,0))
  inp = std * inp + mean
  inp = np.clip(inp, 0, 1) #rounding
  plt.imshow(inp)
  if title is not None:
    plt.title(title)
  plt.axis('off')

inputs, classes = next(iter(data_loader['train']))
out = torchvision.utils.make_grid(inputs)
imshow(out) #, title=[class_names[x] for x in classes])

In [ ]:
#Hyper parameters
num_epochs = 35
num_classes = 2
batch_size = 32
learning_rate = 0.001


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
#model
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module): 
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(720, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [ ]:
model1 = CNN()

In [ ]:
print(model1)

In [ ]:
#create model

def create_model(n_classes):
  model = models.resnet34(pretrained=False)
  n_features = model.fc.in_features
  model.fc = nn.Linear(n_features, n_classes)

  return model.to(device)

In [ ]:
base_model = create_model(2)

In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for inputs, labels in data_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, pred = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)

    correct_predictions += torch.sum(pred == labels )
    losses.append(loss.item())

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  scheduler.step()

  return correct_predictions.double() / n_examples , np.mean(losses)    

In [ ]:
#evaluate function
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()

  losses = []
  correct_predictions = 0

  with torch.no_grad():
    for inputs, labels in data_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)

      _, pred = torch.max(outputs, dim=1)

      loss = loss_fn(outputs, pred)

      correct_predictions += torch.sum(pred == labels)
      losses.append(loss.item())
  return correct_predictions.double() / n_examples , np.mean(losses)

In [ ]:
def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)

In [ ]:
#training 

def train_model(model, data_loader, dataset_sizes, device, n_epochs = 10):
  optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)
  scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma = 0.1)
  loss_fn = nn.CrossEntropyLoss().to(device)

  history = defaultdict(list)
  best_accuracy = 0

  for epoch in range(n_epochs):
    print(f'Epoch {epoch + 1}/ {n_epochs}')
    print('-' * 10)
    train_acc, train_loss = train_epoch(model, data_loader['train'],loss_fn, optimizer, device,scheduler,dataset_sizes['train'] )

    print(f'Train Loss : {round_tensor(train_loss,3)} Accuracy : {round_tensor(train_acc,3)}')

    val_acc , val_loss =  eval_model(model , data_loader['valid'], loss_fn, device, dataset_sizes['valid'])

    print(f'Valid Loss : {round_tensor(val_loss,3)} Accuracy : {round_tensor(val_acc,3)}')
    print()
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['valid_acc'].append(val_acc)
    history['valid_loss'].append(val_loss)

    if val_acc > best_accuracy:
      torch.save(model.state_dict(), 'best_model_state.bin')
      best_accuracy = val_acc
  print(f'Best Accuracy : {round_tensor(best_accuracy,3)}')
  model.load_state_dict(torch.load('best_model_state.bin'))

  return model, history 



In [ ]:
%%time

base_model , history = train_model(base_model, data_loader, data_sizes, device)

In [ ]:
#prediction

def get_predictions(model, data_loader):
  model = model.eval()
  predictions = []
  real_values = []
  with torch.no_grad():
    for inputs, labels in data_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      outputs = model(inputs)
      _, pred = torch.max(outputs, 1)
      predictions.extend(pred)
      real_values.extend(labels)
  predictions = torch.as_tensor(predictions).cpu()
  real_values = torch.as_tensor(real_values).cpu()
  return predictions, real_values

In [ ]:
y_pred, y_test = get_predictions(base_model, data_loader['test'])

In [ ]:
y_pred

In [ ]:
y_pred2 = y_pred.numpy()

In [ ]:
print(len(y_pred2))

In [ ]:
df_sample['has_cactus'] = y_pred2

In [ ]:
df_sample['has_cactus'].value_counts()

In [ ]:
df_sample.to_csv('submission.csv',index=None)

In [ ]:
ls